In [ ]:
"""
MSGLasso.ipynb

Created on Wed Nov 29 2023

@author: Lukas

This file runs multivariate sparse group lasso on graph dataset properties
to predict model accuracy, following "A Metadata-Driven Approach to Understand Graph Neural Networks"
"""

In [1]:
# install asgl

!pip install asgl

  Preparing metadata (setup.py) ... done
  Created wheel for asgl: filename=asgl-1.0.5-py3-none-any.whl size=16002 sha256=fe104bce724a63851f14bfb4517f08f265cc9c8057737e54c57904552577550b
  Stored in directory: /root/.cache/pip/wheels/4b/ac/16/8caac90091e10a732feb3c240d6dcf472c4b0c7f28d2b96479
Successfully built asgl


In [3]:
# Import required packages
import asgl
import pandas as pd
import numpy as np

In [8]:
# Define parameters grid
lambda1 = (10.0 ** np.arange(-3, 1.51, 0.2)) # 23 possible values for lambda
alpha = np.arange(0, 1, 0.05) # 20 possible values for alpha

# Define model parameters
model = 'lm'  # linear model
penalization = 'sgl'  # sparse group lasso penalization
parallel = True  # Code executed in parallel
error_type = 'MSE'  # Error measuremente considered. MSE stands for Mean Squared Error.

In [9]:
# Define a cross validation object
cv_class = asgl.CV(model=model, penalization=penalization, lambda1=lambda1, alpha=alpha,
                   nfolds=5, error_type=error_type, parallel=parallel, random_state=99)

# Compute error using k-fold cross validation
error = cv_class.cross_validation(x=x, y=y, group_index=group_index)

# Obtain the mean error across different folds
error = np.mean(error, axis=1)

# Select the minimum error
minimum_error_idx = np.argmin(error)

# Select the parameters associated to mininum error values
optimal_parameters = cv_class.retrieve_parameters_value(minimum_error_idx)
optimal_lambda = optimal_parameters.get('lambda1')
optimal_alpha = optimal_parameters.get('alpha')

/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py:1387: UserWarn

In [10]:
# Define asgl class using optimal values
asgl_model = asgl.ASGL(model=model, penalization=penalization, lambda1=optimal_lambda, alpha=optimal_alpha)

# Split data into train / test
train_idx, test_idx = asgl.train_test_split(nrows=x.shape[0], train_pct=0.7, random_state=1)

# Solve the model
asgl_model.fit(x=x[train_idx, :], y=y[train_idx], group_index=group_index)

# Obtain betas
final_beta_solution = asgl_model.coef_[0]

# Obtain predictions
final_prediction = asgl_model.predict(x_new=x[test_idx, :])

# Obtain final errors
final_error = asgl.error_calculator(y_true=y[test_idx], prediction_list=final_prediction, error_type=error_type)

**Record Graph Properties**

In [ ]:
# load in the data

mutag = list(TUDataset(root="data", name="MUTAG"))
enzymes = list(TUDataset(root="data", name="ENZYMES"))
proteins = list(TUDataset(root="data", name="PROTEINS"))
imdb = list(TUDataset(root="data", name="IMDB-BINARY"))

In [ ]:
# for each dataset, create a dataframe with the graph index and the properties, 
# i.e. edge density, average degree, degree assortativity, pseudo diameter, average clustering coefficient,
# transitivity, algebraic connectivity, curvature gap, and relative size of the largest clique

def compute_edge_density(data):
    densities = []
    for i in tqdm.tqdm(range(len(data))):
        edge_density = data[i].num_edges / (data[i].num_nodes * (data[i].num_nodes - 1) / 2)
        densities.append(edge_density)
    return densities

def compute_average_degree(data):
    average_degrees = []
    for i in tqdm.tqdm(range(len(data))):
        average_degree = data[i].num_edges / data[i].num_nodes
        average_degrees.append(average_degree)
    return average_degrees

def compute_degree_assortativity(data):
    degree_assortativities = []
    for i in tqdm.tqdm(range(len(data))):
        G = to_networkx(data[i])
        degree_assortativity = nx.degree_assortativity_coefficient(G)
        degree_assortativities.append(degree_assortativity)
    return degree_assortativities

def compute_pseudo_diameter(data):
    pseudo_diameters = []
    for i in tqdm.tqdm(range(len(data))):
        G = to_networkx(data[i])
        G = G.to_undirected()
        if nx.is_connected(G) == False:
            connected_components = list(nx.connected_components(G)).sort(key=len, reverse=True)
            if connected_components is not None:
                G = G.subgraph(connected_components[0])
                pseudo_diameter = nx.algorithms.distance_measures.diameter(G)
            else:
                pseudo_diameter = 1
        else:
            pseudo_diameter = nx.algorithms.distance_measures.diameter(G)
        pseudo_diameters.append(pseudo_diameter)
    return pseudo_diameters

def compute_average_clustering_coefficient(data):
    average_clustering_coefficients = []
    for i in tqdm.tqdm(range(len(data))):
        G = to_networkx(data[i])
        average_clustering_coefficient = nx.average_clustering(G)
        average_clustering_coefficients.append(average_clustering_coefficient)
    return average_clustering_coefficients

def compute_transitivity(data):
    transitivities = []
    for i in tqdm.tqdm(range(len(data))):
        G = to_networkx(data[i])
        G.to_undirected()
        transitivity = nx.transitivity(G) 
        transitivities.append(transitivity)
    return transitivities

def compute_algebraic_connectivity(data):
    connectivities = []
    for i in tqdm.tqdm(range(len(data))):
        G = to_networkx(data[i]).to_undirected()
        algebraic_connectivity = nx.algebraic_connectivity(G)
        connectivities.append(algebraic_connectivity)
    return connectivities

def compute_curvature_gap(data):
    curvature_gaps = []
    for i in tqdm.tqdm(range(len(data))):
        try:
            G = to_networkx(data[i])
            rc = OllivierRicci(G, alpha=0.5, verbose="ERROR")
            rc.compute_ricci_curvature()

            curvature = np.array(list(rc.G.nodes(data="ricciCurvature")))
            curvature = curvature[:, 1].reshape(-1, 1)
            gmm = GaussianMixture(n_components=2, covariance_type="full", random_state=0).fit(curvature)

            curvature_gap = abs(gmm.means_[0] - gmm.means_[1]) / np.sqrt(0.5 * (gmm.covariances_[0] ** 2 + gmm.covariances_[1] ** 2))
            curvature_gaps.append(curvature_gap)
        except:
            curvature_gaps.append(0)
    return curvature_gaps

def compute_max_clique(data):
    max_cliques = []
    for i in tqdm.tqdm(range(len(data))):
        G = to_networkx(data[i]).to_undirected()
        max_clique = nx.algorithms.clique.graph_clique_number(G)
        max_cliques.append(max_clique)
    return max_cliques
    

# MUTAG
mutag_df = pd.DataFrame(columns=['graph_index', 'edge_density', 'avg_degree', 'degree_assortativity', 'pseudo_diameter', 'avg_clustering_coeff', 'transitivity', 'algebraic_connectivity', 'curvature_gap', 'rel_size_largest_clique'])
mutag_df['graph_index'] = range(len(mutag))
mutag_df['edge_density'] = compute_edge_density(mutag)
mutag_df['avg_degree'] = compute_average_degree(mutag)
mutag_df['degree_assortativity'] = compute_degree_assortativity(mutag)
mutag_df['pseudo_diameter'] = compute_pseudo_diameter(mutag)
mutag_df['avg_clustering_coeff'] = compute_average_clustering_coefficient(mutag)
mutag_df['transitivity'] = compute_transitivity(mutag)
mutag_df['algebraic_connectivity'] = compute_algebraic_connectivity(mutag)
mutag_df['curvature_gap'] = compute_curvature_gap(mutag)
mutag_df['rel_size_largest_clique'] = compute_max_clique(mutag)

# ENZYMES
enzymes_df = pd.DataFrame(columns=['graph_index', 'edge_density', 'avg_degree', 'degree_assortativity', 'pseudo_diameter', 'avg_clustering_coeff', 'transitivity', 'algebraic_connectivity', 'curvature_gap', 'rel_size_largest_clique'])
enzymes_df['graph_index'] = range(len(enzymes))
enzymes_df['edge_density'] = compute_edge_density(enzymes)
enzymes_df['avg_degree'] = compute_average_degree(enzymes)
enzymes_df['degree_assortativity'] = compute_degree_assortativity(enzymes)
enzymes_df['pseudo_diameter'] = compute_pseudo_diameter(enzymes)
enzymes_df['avg_clustering_coeff'] = compute_average_clustering_coefficient(enzymes)
enzymes_df['transitivity'] = compute_transitivity(enzymes)
enzymes_df['algebraic_connectivity'] = compute_algebraic_connectivity(enzymes)
enzymes_df['curvature_gap'] = compute_curvature_gap(enzymes)
enzymes_df['rel_size_largest_clique'] = compute_max_clique(enzymes)

# PROTEINS
proteins_df = pd.DataFrame(columns=['graph_index', 'edge_density', 'avg_degree', 'degree_assortativity', 'pseudo_diameter', 'avg_clustering_coeff', 'transitivity', 'algebraic_connectivity', 'curvature_gap', 'rel_size_largest_clique'])
proteins_df['graph_index'] = range(len(proteins))
proteins_df['edge_density'] = compute_edge_density(proteins)
proteins_df['avg_degree'] = compute_average_degree(proteins)
proteins_df['degree_assortativity'] = compute_degree_assortativity(proteins)
proteins_df['pseudo_diameter'] = compute_pseudo_diameter(proteins)
proteins_df['avg_clustering_coeff'] = compute_average_clustering_coefficient(proteins)
proteins_df['transitivity'] = compute_transitivity(proteins)
proteins_df['algebraic_connectivity'] = compute_algebraic_connectivity(proteins)
proteins_df['curvature_gap'] = compute_curvature_gap(proteins)
proteins_df['rel_size_largest_clique'] = compute_max_clique(proteins)

# IMDB
imdb_df = pd.DataFrame(columns=['graph_index', 'edge_density', 'avg_degree', 'degree_assortativity', 'pseudo_diameter', 'avg_clustering_coeff', 'transitivity', 'algebraic_connectivity', 'curvature_gap', 'rel_size_largest_clique'])
imdb_df['graph_index'] = range(len(imdb))
imdb_df['edge_density'] = compute_edge_density(imdb)
imdb_df['avg_degree'] = compute_average_degree(imdb)
imdb_df['degree_assortativity'] = compute_degree_assortativity(imdb)
imdb_df['pseudo_diameter'] = compute_pseudo_diameter(imdb)
imdb_df['avg_clustering_coeff'] = compute_average_clustering_coefficient(imdb)
imdb_df['transitivity'] = compute_transitivity(imdb)
imdb_df['algebraic_connectivity'] = compute_algebraic_connectivity(imdb)
imdb_df['curvature_gap'] = compute_curvature_gap(imdb)
imdb_df['rel_size_largest_clique'] = compute_max_clique(imdb)

In [ ]:
# save all dataframes to csv files
mutag_df.to_csv('mutag.csv')
enzymes_df.to_csv('enzymes.csv')
proteins_df.to_csv('proteins.csv')
imdb_df.to_csv('imdb.csv')

In [ ]:
# load in the accuracy dictionaryies and add them to the dataframes

# MUTAG
with open('results/mutag_graph_dict.pickle', 'rb') as handle:
    mutag_accs = pickle.load(handle)

mutags_avgs = {}
for key in mutag_accs.keys():
    mutags_avgs[key] = np.mean(mutag_accs[key])

mutag_df['accuracy'] = mutag_df['graph_index'].map(mutags_avgs)

# ENZYMES
with open('results/enzymes_graph_dict.pickle', 'rb') as handle:
    enzymes_accs = pickle.load(handle)

enzymes_avgs = {}
for key in enzymes_accs.keys():
    enzymes_avgs[key] = np.mean(enzymes_accs[key])

enzymes_df['accuracy'] = enzymes_df['graph_index'].map(enzymes_avgs)

# PROTEINS
with open('results/proteins_graph_dict.pickle', 'rb') as handle:
    proteins_accs = pickle.load(handle)

proteins_avgs = {}
for key in proteins_accs.keys():
    proteins_avgs[key] = np.mean(proteins_accs[key])

proteins_df['accuracy'] = proteins_df['graph_index'].map(proteins_avgs)

# IMDB
with open('results/imdb_graph_dict.pickle', 'rb') as handle:
    imdb_accs = pickle.load(handle)

imdb_avgs = {}
for key in imdb_accs.keys():
    imdb_avgs[key] = np.mean(imdb_accs[key])

imdb_df['accuracy'] = imdb_df['graph_index'].map(imdb_avgs)

**Regression**

In [ ]:
# set hyperparameters

alpha_opt = 0.5
lambda_opt = 0.002

In [ ]:
# run regression with accuracy as the target variable and the properties as the features

# MUTAG
x = mutag_df[['edge_density', 'avg_degree', 'degree_assortativity', 'pseudo_diameter', 'avg_clustering_coeff', 'transitivity', 'algebraic_connectivity', 'curvature_gap', 'rel_size_largest_clique']].to_numpy()
y = mutag_df['accuracy'].to_numpy()
group_index = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1])

# standardize the data by subtracting the mean and dividing by the standard deviation
x_standardized = (x - np.mean(x, axis=0)) / np.std(x, axis=0)

# fit the model

model = asgl.ASGL(model='lm', penalization='sgl', lambda1=lambda_opt, alpha=alpha_opt)
model.fit(x=x_standardized, y=y, group_index=group_index)


In [50]:
# get the coefficients
betas = model.coef_[0]

array([ 2.64046511e+02,  0.00000000e+00,  1.24063100e+00,  4.09944460e+00,
       -1.23699372e+01,  1.18208572e+02,  2.70692476e+01,  3.44738482e+01,
       -1.86345734e-01, -4.87576259e+01,  2.13417824e+02,  0.00000000e+00,
       -1.42752151e+02,  9.38657064e+01,  1.02056033e+02, -2.46636576e+02])

In [ ]:
# get the predictions
predictions = model.predict(x_new=x_standardized)

# get the errors
error = asgl.error_calculator(y_true=y, prediction_list=predictions, error_type='MSE')

# plot the predicted accuracy and the actual accuracy on each graph in MUTAG
plt.scatter(range(len(predictions)), predictions, label='Predicted Accuracy')
plt.scatter(range(len(predictions)), y, label='Actual Accuracy')
plt.xlabel('Graph Index')
plt.ylabel('Accuracy')
plt.legend()
plt.title('MUTAG Accuracy Predictions')
plt.show()